In [1]:
%pwd

'/workspaces/chicken-disease-classification/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/workspaces/chicken-disease-classification'

## Entities 

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

## configurtions manager in src/config

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

2023-09-22 13:03:44.220685: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 13:03:44.249682: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-22 13:03:44.250600: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 13:03:44.972515: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [8]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        print("Configuration Manager Initiated")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        
        create_directories([Path(self.config.prepare_callbacks.root_dir),
                            Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
                            Path(self.config.prepare_callbacks.checkpoint_model_filepath)])
        return PrepareCallbacksConfig(
            root_dir = Path(self.config.prepare_callbacks.root_dir),
            tensorboard_root_log_dir = Path(self.config.prepare_callbacks.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(self.config.prepare_callbacks.checkpoint_model_filepath)
        )
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = self.config.data_ingestion.unzip_dir
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


## Components

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf # to call VGG16 from keras
import time

In [10]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
    
    

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        self.model = self.get_base_model()

    def get_base_model(self):
        model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        model.compile(optimizer='adam',
                        loss='categorical_crossentropy'
                        ,metrics=['accuracy'])
        return model


    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.models.Model):
        model.save(path)

    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


## Pipeline

In [12]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
except Exception as e:
    raise e


Configuration Manager Initiated
[ 2023-09-22 13:04:07,763: INFO: common:  yaml file: config/config.yaml loaded successfully]
[ 2023-09-22 13:04:07,766: INFO: common:  yaml file: params.yaml loaded successfully]
[ 2023-09-22 13:04:07,767: INFO: common:  directory: artifacts created successfully]
[ 2023-09-22 13:04:07,767: INFO: common:  directory: artifacts created successfully]
[ 2023-09-22 13:04:07,768: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-22 13:04:07,769: INFO: common:  directory: artifacts/prepare_callbacks created successfully]
[ 2023-09-22 13:04:07,770: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-22 13:04:07,770: INFO: common:  directory: artifacts/prepare_callbacks/tensorboard_log_dir created successfully]
[ 2023-09-22 13:04:07,772: INFO: common:  directory: artifacts/prepare_callbacks/checkpoint_dir created successfully]
[ 2023-09-22 13:04:07,773: INFO: common:  directo

2023-09-22 13:04:11.316708: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-09-22 13:04:11.380064: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-09-22 13:04:11.800342: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-09-22 13:04:11.964205: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.


 1/40 [..............................] - ETA: 2:16 - loss: 0.7906 - accuracy: 0.2500

2023-09-22 13:04:14.210851: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


40/40 [==============================] - 155s 4s/step - loss: 0.2776 - accuracy: 0.8656 - val_loss: 0.1779 - val_accuracy: 0.9438
